<a href="https://colab.research.google.com/github/JoaoAlexandreFerreira/PINNs/blob/main/In%C3%ADcio_Pinn_inversa_Burgers_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Inverse PINNs - Burgers

Bibliotecas

In [ ]:
import tensorflow as tf
from scipy.io import loadmat
import numpy as np
import keras
import matplotlib.pyplot as plt
import pandas as pd
from keras import Sequential
from keras.layers import Input, Dense
from time import time
from keras.optimizers import Adam
from keras.initializers import Ones, GlorotNormal, he_normal, Zeros
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy.optimize
keras.backend.set_floatx('float32')
tf.random.set_seed(0)

#Descrição do Problema

Criação de uma rede neural para problema inverso da equação de Burger, conforme:

  $\frac{\partial u}{\partial t} + λ_1u\frac{\partial u}{\partial x} = λ_2\frac{\partial^2 u}{\partial x^2}$

  $ x \in [-1,1]$

  $ t \in [0,1] $

  com os valores reais de $λ_1 = 1$ e $\lambda _2 = 𝜈 = \frac{0.01}{π}$

  **Em problemas diretos**, tinha-se: Modelos → Dados (previsão)

  Já em **problemas inversos**, têm-se: Dados → Modelo (obter os parâmetros do modelo)

#PINNS para problemas inversos -- "Data-driven Discovery of Nonlinear Partial Differential Equations"

**Problemas Inversos**: Dados → Parâmetros do modelo:

Dados → PINN → Parâmetros do modelo, que neste caso são os parâmetros da Equação diferencial parcial

Em Raissi *et al*, (2017) é dito que a forma geral das equações diferenciais parciais não lineares e parametrizadas é:

$u_t + 𝞕[u;λ] = 0$

com $u_t$ sendo a solução e $ 𝞕[u;λ]$ um operador não linear parametrizado por $λ$.

Ou seja, a PINN é utilizada para obter λ.

###Pelo teorema da aproximação universal, as redes neurais podem ser utilizadas para aproximar qualquer função. (https://book.sciml.ai/notes/03/)

Sendo a rede neural descrita pela equação:

$N(X) = W_nσ_{n-1}(W_{n-1}σ_{n-2}(...(W_2σ(W_1X + b_1) + b_2) + ...) + b_{n-1}) + b_n$

Do teorema podemos assumir então que:

$N(x,t) ≈ u(x,t)$

Logo:

$N_t + λ_1NN_x - λ_2N_{xx} ≈ u_t + λ_1uu_x - λ_2u_{xx} = 0$

A função $f$ é portanto:

$f(x,t) = N_t + λ_1NN_x - λ_2N_{xx} = N_t + Ζ[N, λ] ≈ 0$

###Custo - Loss

Para a pinn inversa, é definido um certo número de pontos dentro do domínio ($N_u$) o qual é verificado o valor da função $f$ nesses pontos, e em seguida minimizado o erro associado ao valor predito e o real, por meio da equação:

$MSE_f = \frac{1}{N_u}∑^{N_u}_{i=1}|f(t_u^i, x_u^i)|^2$

Por se tratar de um problema inverso, os dados são conhecidos, assim, o segundo custo é da forma:

$MSE_u = \frac{1}{N_u}∑^{N_u}_{i=1}|u(t_u^i, x_u^i) - N(t^i_u, x_u^i)|^2$

$MSE_{total} = MSE_f + MSE_u$

Parâmetros

In [ ]:
pi = tf.constant(np.pi)
mi = 0.01/tf.constant(np.pi)
dados = loadmat('burgers_shock.mat')
x_dados = dados['x'].flatten()[:,None]
t_dados = dados['t'].flatten()[:,None]
u_dados = np.real(dados['usol']).T

X, T = np.meshgrid(x_dados, t_dados)


#Definindo os pontos X
N_0 = 100 #100 pontos para condição inicial
N_b = 100 #100 pontos para condição de contorno
N_r = 10000 #Pontos para a edp

#Pontos do domínio, dado pelo Maziar
tmin = 0. ; tmax = 1.
xmin = -1.; xmax = 1.
X_real = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
U_real = u_dados.flatten('F')[:,None]

lb = tf.constant(X_real[0], dtype = 'float32'); ub = tf.constant(X_real[-1], dtype = 'float32')

lambda1 = tf.Variable(initial_value = 5., trainable = True)
lambda2 = tf.Variable(initial_value = 0.001, trainable = True)

In [ ]:
def inicial(x):
  return -tf.sin(pi*x)

def contorno(t, x):
  n = x.shape[0]
  return tf.zeros((n,1))

#Obtendo pontos para a condição inicial
t0 = tf.zeros((N_0,1))*lb[1].numpy()
x0 = tf.random.uniform((N_0,1), lb[1].numpy(), ub[1].numpy()) #Colocando os valores de x0 em ordem aleatoria, indo de -1 a 1
x0 = tf.concat([t0, x0], 1) #Criando uma matriz com os valores de tempo = 0 e de x0

#Valores de u para a condição inicial
u_ini = inicial(x0[:,1:2])

#Repetindo o processo, mas para a condição de contorno
tb = tf.random.uniform((N_b,1), lb[0].numpy(), ub[0].numpy())
xb = lb[1].numpy() + (ub[1].numpy() - lb[1].numpy()) * tf.keras.backend.random_bernoulli((N_b,1), 0.5)
xb = tf.concat([tb, xb], 1)

#Valores na condição de contorno
u_cont = contorno(xb[:,0:1], xb[:,1:2])

#Repetindo o processo, mas agora é para obter os pontos da EDP
tr = tf.random.uniform((N_r,1), lb[0].numpy(), ub[0].numpy())
xr = tf.random.uniform((N_r,1), lb[1].numpy(), ub[1].numpy())
xr = tf.concat([tr, xr], 1)

#Fazendo uma lista, para uso posterior
X_cond = [x0, xb]
u_cond = [u_ini, u_cont]

In [ ]:
qtd_pontos = len(x_dados)*len(t_dados)
id = np.random.choice(qtd_pontos, N_r, replace=False)
X_treino = X_real[id]
U_treino = U_real[id]

In [ ]:
print("Dos", qtd_pontos, "dados fornecidos serão usados", X_treino.shape[0])

Dos 25600 dados fornecidos serão usados 10000


In [ ]:
#Agora criando o modelo de Rede neural

def modelopinn(nos, camadas_ocultas):
  modelo = Sequential()
  #Inserindo o numero de variaveis de entrada
  modelo.add(Input(shape = (2,)))
  #Escala de entrada, mapeando os pontos de maximo e minimo
  modelo.add(keras.layers.Lambda(
      lambda x: 2.0*(x - lb)/(ub - lb) - 1.0
  ))
  for i in range(camadas_ocultas):
        modelo.add(Dense(nos, activation='tanh', kernel_initializer=GlorotNormal())) #camada oculta



  modelo.add(Dense(1)) #camada de saída
  lambda1 = modelo.add_weight(name='lambda1', initializer=tf.constant_initializer(5.0), trainable=True)
  lambda2 = modelo.add_weight(name='lambda2', initializer=tf.constant_initializer(0.1), trainable=True)
  modelo.summary()
  return modelo, lambda1, lambda2

In [ ]:
#modelo, lambda1, lambda2 = modelopinn(20,8)
modelo, lambda1, lambda2 = modelopinn(20,8)

otimizador = Adam(learning_rate = 0.001)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                      │ (None, 2)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 20)                  │              60 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,023 (11.81 KB)

 Trainable params: 3,023 (11.81 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print("O lambda 1 real é 1 e o lambda 2 real é", 0.01/pi, "lambda 1 estimado foi: ", lambda1.numpy(), "lambda 2 estimado foi:", lambda2.numpy())

O lambda 1 real é 1 e o lambda 2 real é tf.Tensor(0.0031830987, shape=(), dtype=float32) lambda 1 estimado foi:  5.0 lambda 2 estimado foi: 0.1


In [ ]:
print("Variáveis treináveis:", [var.name for var in modelo.trainable_variables])

Variáveis treináveis: ['lambda1', 'lambda2', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias']


In [ ]:
#E tirando os gradientes, para calcular a edp
def gradiente(modelo, X_r):
  with tf.GradientTape(persistent=True) as tape:

    #Registrando tempo e posição para a diferenciação automática
    t, x = X_r[:, 0:1], X_r[:,1:2]
    tape.watch(t)
    tape.watch(x)

    #previsão do modelo
    u = modelo(tf.stack([t[:,0], x[:,0]], 1))
    #gradiente du/dx
    ux = tape.gradient(u, x)


  #gradiente du/dt
  ut = tape.gradient(u, t)
  #gradiente du²/d²x
  uxx = tape.gradient(ux, x)

  del tape

  return ut + lambda1*u*ux - lambda2*uxx

def MSE(modelo, xr, X_cond, u_cond):

    #Erro edp
    r = gradiente(modelo, xr)
    erro = tf.reduce_mean(tf.square(r))

    loss = erro

    #Erro da rede neural
    for i in range(len(X_cond)):
        u_pred = modelo(X_cond[i])
        loss += tf.reduce_mean(tf.square(u_cond[i] - u_pred))

    return erro, loss

def grad(modelo, xr, X_cond, u_cond):
  #tirando o gradiente em relação ao modelos, para que eles sejam atualizados posteriormente
  with tf.GradientTape(persistent=True) as tape:
    #tape.gradient(modelo.trainable_variables)
    erro, loss = MSE(modelo, xr, X_cond, u_cond)

  g = tape.gradient(loss, modelo.trainable_variables)
  del tape

  return erro, loss, g

In [ ]:
#etapa de treinamento como uma função do TensorFlow para aumentar a velocidade do treinamento
@tf.function
def train_step(modelo):
  #Calculando a perda do modelo em relação ao modelo, com a função grad
  erro, loss, grad_theta = grad(modelo, xr, X_cond, u_cond)

  #Aplicando o gradiente as variaveis do modelo de rede neural
  otimizador.apply_gradients(zip(grad_theta, modelo.trainable_variables))

  return erro, loss

itr = 5000
historico = []
erro_aux = []
t0 = time()

for i in range(itr+1):

    erro, loss = train_step(modelo)

    #Salvando os erros para listar
    historico.append(loss.numpy())
    erro_aux.append(erro.numpy)

    if i%10 == 0:
        print(i,"Loss treino: {:10.8e}, Loss edp: {:10.8e}, Lambda1: {:1.1e}, Lambda2: {:1.1e}".format(loss, erro, lambda1.numpy(), lambda2.numpy()))

print('\nTempo de treino da rede neural: {} segundos'.format(time()-t0))

0 Loss treino: 5.08220136e-01, Loss edp: 8.06579366e-03, Lambda1: 5.0e+00, Lambda2: 9.9e-02
10 Loss treino: 2.88259327e-01, Loss edp: 1.91083015e-03, Lambda1: 5.0e+00, Lambda2: 9.6e-02
20 Loss treino: 2.80728102e-01, Loss edp: 3.79672204e-03, Lambda1: 5.0e+00, Lambda2: 9.7e-02
30 Loss treino: 2.61018574e-01, Loss edp: 2.13662256e-03, Lambda1: 5.0e+00, Lambda2: 1.0e-01
40 Loss treino: 2.29525119e-01, Loss edp: 4.49439045e-03, Lambda1: 5.0e+00, Lambda2: 1.1e-01
50 Loss treino: 1.89789772e-01, Loss edp: 1.28850313e-02, Lambda1: 5.0e+00, Lambda2: 1.1e-01
60 Loss treino: 1.65807575e-01, Loss edp: 3.62853259e-02, Lambda1: 5.0e+00, Lambda2: 9.7e-02
70 Loss treino: 1.54401794e-01, Loss edp: 4.15884778e-02, Lambda1: 5.0e+00, Lambda2: 8.0e-02
80 Loss treino: 1.47056535e-01, Loss edp: 3.17831412e-02, Lambda1: 5.0e+00, Lambda2: 6.9e-02
90 Loss treino: 1.40068531e-01, Loss edp: 2.65087001e-02, Lambda1: 5.0e+00, Lambda2: 6.7e-02
100 Loss treino: 1.33572474e-01, Loss edp: 2.48591416e-02, Lambda1: 5.0

KeyboardInterrupt: 